In [1]:
"""
This script runs an evaluation of a LangGraph agent on a dataset using LangSmith's async evaluation API.
It converts dataset examples into the agent's internal state format, invokes the agent graph with unique checkpointing,
and uses a custom evaluator that leverages an LLM judge to compare the agent's output against reference answers.

Key components:
- example_to_state: converts dataset inputs to the agent's state format.
- target_with_config: async wrapper to invoke the agent graph with checkpoint config.
- correct: async evaluator function that prompts an LLM to judge correctness of the agent's answer.
- aevaluate: runs the evaluation over the dataset with concurrency and experiment tracking.
"""

#==============================================================================
# IMPORTS
#==============================================================================
import sys
import os

# Add the parent directory to the Python path so we can import the my_agent module
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import uuid
from langsmith import aevaluate
from my_agent import create_graph
from my_agent.utils.state import DataAnalysisState
from my_agent.utils.models import get_azure_llm

#==============================================================================
# INITIALIZATION
#==============================================================================
# Get Model
judge_llm = get_azure_llm(temperature=0.0)

# Create the agent graph
graph = create_graph()

#==============================================================================
# EVALUATION FUNCTIONS
#==============================================================================
async def correctness(outputs: dict, reference_outputs: dict) -> bool:
    """
    Evaluator function that uses an LLM to determine if the actual answer contains all information
    from the expected answer.

    Args:
        outputs (dict): The output dictionary from the agent, expected to contain a 'messages' list.
        reference_outputs (dict): The reference outputs from the dataset, expected to have 'answers' key.

    Returns:
        bool: True if the LLM judge determines the answer is correct, False otherwise.
    """
    if not outputs or "messages" not in outputs or not outputs["messages"]:
        return False
        
    actual_answer = outputs["messages"][-1].content
    expected_answer = reference_outputs.get("answers", "[NO EXPECTED ANSWER PROVIDED]")
    
    instructions = (
        "Given an actual answer and an expected answer, determine whether"
        " the actual answer contains all of the information in the"
        " expected answer. Respond with 'CORRECT' if the actual answer"
        " does contain all of the expected information and 'INCORRECT'"
        " otherwise. Do not include anything else in your response."
    )
    user_msg = f"ACTUAL ANSWER: {actual_answer}\n\nEXPECTED ANSWER: {expected_answer}"
    
    response = await judge_llm.ainvoke(
        [
            {"role": "system", "content": instructions},
            {"role": "user", "content": user_msg}
        ]
    )
    return response.content.upper() == "CORRECT"

#==============================================================================
# STATE CONVERSION FUNCTIONS
#==============================================================================
def example_to_state(inputs: dict) -> dict:
    """
    Converts dataset example inputs into the agent's internal DataAnalysisState format.

    Args:
        inputs (dict): Input dictionary from the dataset, expected to have 'question' key.

    Returns:
        DataAnalysisState: Initialized state object for the agent.
    """
    return DataAnalysisState(
        prompt=inputs['question'],
        messages=[],
        result="",
        iteration=0
    )

async def target_with_config(inputs: dict):
    """
    Async wrapper to invoke the agent graph with a unique checkpoint thread_id in the config.

    Args:
        inputs (dict): Input dictionary from the dataset.

    Returns:
        dict: The output from the agent graph invocation, including 'messages'.
    """
    state = example_to_state(inputs)
    config = {"configurable": {"thread_id": str(uuid.uuid4())}}
    return await graph.ainvoke(state, config=config)

#==============================================================================
# MAIN EVALUATION
#==============================================================================
experiment_results = await aevaluate(
    target_with_config,
    data="czsu agent",
    evaluators=[correctness],
    max_concurrency=4,
    experiment_prefix="test1",
)

print(f"Evaluation results: {experiment_results}")


c:\Users\retko\anaconda3\envs\prototype4a\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'test1-7e34d5b8' at:
https://smith.langchain.com/o/817120e4-bc86-484a-9c30-795e42f5bb9e/datasets/e585a306-7113-491d-ba41-bc03c09eb612/compare?selectedSessions=100ac580-59bf-4fd8-94e3-38f0e662a1a2




0it [00:00, ?it/s]

AAAAAAAAAAAAAA
3: Enter get_schema_node
3: Schema details appended
AAAAAAAAAAAAAA
3: Enter get_schema_node
3: Schema details appended
AAAAAAAAAAAAAA
3: Enter get_schema_node
3: Schema details appended
4: Enter query_node
AAAAAAAAAAAAAA
3: Enter get_schema_node
3: Schema details appended
4: Enter query_node
4: Enter query_node
4: Enter query_node
21: =====================================
21: Executing query:
21: SELECT MAX(value) AS max_female_population, "ČR, kraje" AS region FROM OBY01PDT01 WHERE "Ukazatel" = 'Počet obyvatel na počátku období - ženy' GROUP BY "ČR, kraje" ORDER BY max_female_population DESC LIMIT 1
21: =====================================
21: Query result:
21: [(5557945, 'Česko')]
21: =====================================
4: Successfully executed query: SELECT MAX(value) AS max_female_population, "ČR, kraje" AS region FROM OBY01PDT01 WHERE "Ukazatel" = 'Počet obyvatel na počátku období - ženy' GROUP BY "ČR, kraje" ORDER BY max_female_population DESC LIMIT 1
4: Query r

1it [00:05,  5.58s/it]

...

11: LLM decision: 'query_again'
11: Routing to generate another query
10: Analysis completed
10: Formatted input sent to LLM:
10: Question: Find records where male count equals female count
10: Results provided:
Query 1:
SELECT * FROM OBY01PDT01 WHERE "Počet obyvatel na počátku období - muži" = "Počet obyvatel na počátku období - ženy" LIMIT 5
Result 1:
No results found
7: Enter submit_final_answer_node
7: Final answer prepared
4: Enter query_node
8: Enter save_node
8: ✅ Result saved to E:\OneDrive\Knowledge Base\0207_GenAI\Code\LangGraph\testing_prototypes\prototype4\analysis_results.txt and E:\OneDrive\Knowledge Base\0207_GenAI\Code\LangGraph\testing_prototypes\prototype4\analysis_results.json


29it [03:15,  3.80s/it]

21: =====================================
21: Executing query:
21: SELECT value FROM OBY01PDT01 WHERE "ČR, kraje" = 'Jihomoravský kraj' AND "Ukazatel" = 'Počet obyvatel na počátku období - ženy' AND "Kumulace čtvrtletí" = 'Q1-Q3 2024' LIMIT 5
21: =====================================
21: Query result:
21: 626208
21: =====================================
4: Successfully executed query: SELECT value FROM OBY01PDT01 WHERE "ČR, kraje" = 'Jihomoravský kraj' AND "Ukazatel" = 'Počet obyvatel na počátku období - ženy' AND "Kumulace čtvrtletí" = 'Q1-Q3 2024' LIMIT 5
4: Query result: 626208
4: Current state of queries_and_results: [('SELECT value FROM OBY01PDT01 WHERE "ČR, kraje" = \'Jihomoravský kraj\' AND "Ukazatel" = \'Střední stav obyvatel - ženy\' AND "Kumulace čtvrtletí" = \'Q1-Q3 2024\' LIMIT 5', '625214'), ('SELECT value FROM OBY01PDT01 WHERE "ČR, kraje" = \'Jihomoravský kraj\' AND "Ukazatel" = \'Střední stav obyvatel - ženy\' AND "Kumulace čtvrtletí" = \'Q1-Q3 2024\' LIMIT 5', '625214')

30it [03:19,  6.65s/it]

Evaluation results: <AsyncExperimentResults test1-7e34d5b8>
